In [5]:
import requests
import pandas as pd
import time
import os
import sys

# Add the 'scripts' folder to the Python path
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from discord_notifier import send_discord_message

# Ensure the directory exists
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to retrieve PUUIDs from summoner_details CSV
def retrieve_puuids(csv_file='../data/processed/summoner_details.csv'):
    # Read the summoner details file
    df_summoner = pd.read_csv(csv_file)

    # Filter players who don't have 60 match IDs in match_ids.csv
    if os.path.exists('../data/processed/match_ids.csv'):
        df_match_ids = pd.read_csv('../data/processed/match_ids.csv')
        puuid_match_counts = df_match_ids.groupby('puuid').size()
        puuids_to_fetch = df_summoner[df_summoner['puuid'].isin(puuid_match_counts[puuid_match_counts < 60].index)]['puuid'].tolist()
    else:
        puuids_to_fetch = df_summoner['puuid'].tolist()

    return puuids_to_fetch

# Function to append match ID-PUUID pairs to match_ids CSV
def insert_match_ids(match_id_puuid_pairs, csv_file='../data/processed/match_ids.csv'):
    # Check if the file exists
    if os.path.exists(csv_file):
        df_match_ids = pd.read_csv(csv_file)
    else:
        # If it doesn't exist, create an empty DataFrame
        df_match_ids = pd.DataFrame(columns=['match_id', 'puuid'])

    # Convert the match_id_puuid_pairs into a DataFrame
    new_data = pd.DataFrame(match_id_puuid_pairs, columns=['match_id', 'puuid'])

    # Append new data and remove duplicates
    df_match_ids = pd.concat([df_match_ids, new_data]).drop_duplicates()

    # Save the updated DataFrame back to CSV
    df_match_ids.to_csv(csv_file, index=False)
    print(f"Inserted {len(new_data)} match ID-PUUID pairs into {csv_file}.")

# Function to pull match IDs from the Riot API
def pull_match_ids(api_key, puuid, base_url, matches_per_player):
    url = f"{base_url}{puuid}/ids?start=0&count={matches_per_player}&api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return [(match_id, puuid) for match_id in response.json()]
    else:
        print(f"Error {response.status_code} fetching matches for {puuid}: {response.text}")
        return []

# Function to manage API rate limits
def manage_api_rate_limit():
    time.sleep(1.3)  # Adjust based on the Riot Games API rate limits

# Main function to pull and store match IDs
def main(matches_per_player=30):
    api_key = "RGAPI-b8762239-29e3-48eb-a6c7-5b9198da9aa1"
    base_url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/"
    players = retrieve_puuids()  # Retrieve only players with fewer than 60 matches
    batch_size = 1000  # Set the threshold for batch processing
    accumulated_pairs = []

    try:
        count = 1
        total = len(players)
        for puuid in players:
            print(f"Fetching: {count} out of {total} - PUUID: {puuid}")
            
            # Check how many matches the player already has
            df_match_ids = pd.read_csv('../data/processed/match_ids.csv')
            current_match_count = df_match_ids[df_match_ids['puuid'] == puuid].shape[0]
            remaining_matches_to_fetch = matches_per_player - current_match_count
            
            # Only fetch matches if fewer than 60 have been fetched
            if remaining_matches_to_fetch > 0:
                match_id_puuid_pairs = pull_match_ids(api_key, puuid, base_url, 60)
                if match_id_puuid_pairs:
                    accumulated_pairs.extend(match_id_puuid_pairs)
                    if len(accumulated_pairs) >= batch_size:
                        insert_match_ids(accumulated_pairs)
                        accumulated_pairs = []  # Reset the list after insertion
                manage_api_rate_limit()
            count += 1
        
        # Insert any remaining pairs that didn't reach the batch size
        if accumulated_pairs:
            insert_match_ids(accumulated_pairs)

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the main function
if __name__ == "__main__":
    try:
        main(matches_per_player=30)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        send_discord_message()


Fetching: 1 out of 2376 - PUUID: 8AwbYwK3VvKk2_RtGM1gTdANRkhoCcuSRu-NWyXTbLEzPlHZBEVU8wPjExt1Ro88Z0i_HQuHb-UTtA
Fetching: 2 out of 2376 - PUUID: eRbbvX41pkkfQyQWMc7cYuwoP9PIgrNRzLn44Q4QbSZLnidRZGzQCaTgfri-powyZ6dyOHdyApPsgg
Fetching: 3 out of 2376 - PUUID: ezCpI4N6QqBs2Ft__XUnG80TvnPdM4T49J-Kcc2rkwXe6AQhAKcon8S15kZns6_yD6nytB4t4g6FGw
Fetching: 4 out of 2376 - PUUID: 0S2pHoFrVHu29-uAB88Bh3i6KyuRL6dMxuUDrrs5MloYjVePciJke4G93aaRhNdpOxUAlk8NS73xGg
Fetching: 5 out of 2376 - PUUID: jS88OXUDsGmaYLcMvwsY8YmzC8RZldDU2ShLg_ZPX2Hy40Ok5J1MFC_CCZ2VWrzWix2cADmLcFbenA
Fetching: 6 out of 2376 - PUUID: KL7_xbYgztT-RRXn2-HwfRoyP14u1d_NNNyciSxJrFG8SYOhkgqqwpxoR0YdOqpdqbinUXCGInbt5w
Fetching: 7 out of 2376 - PUUID: 4e67YE9g9Ya_498w3qisWqetyvVkWVluy0vFhC_cpWCPe-OiPf-Qn61LGtVjkG8Chg2Sf9l72r2k-A
Fetching: 8 out of 2376 - PUUID: QYccaXa8Rj6vLqNt1vwR2qnOcrsWOk-3asqB22WMu3BfZ9nigC5kxrhXZ2Kn9JMydUBah57V6y8Btw
Fetching: 9 out of 2376 - PUUID: XmXKjt26PPURTl14GGhi5DSAJyGQvCaH6-ouz-vRmZNNuNGTOJHZOqMPwREcEcT5sRu5kHo